# 1.Preprocess

- このnotebookはluigiで構築されたscriptのうち、cleansing.pyを、作業の流れが理解しやすいようにnotebook上に移植したもの。
- notebookは試行錯誤に適している一方、中間データからの再開や並列処理は難しい。
- 試行錯誤が終わり、コードがある程度固まってきたら、notebookからscriptに移行することが望ましい。
- script移行の際に、[luigi](https://github.com/spotify/luigi)や[airflow](https://github.com/apache/airflow)などのタスク管理ライブラリを用いると、中間データからの再開や並列処理が容易に実装できる。また、sparkやRDBに対応しているため、ある程度までのスケーラビリティも持たせることができる。
- また、script化を見越したソース管理のフォルダ構成としては、[drivendata社のcookiecutterテンプレート](https://github.com/drivendata/cookiecutter-data-science)等が便利である。本プロジェクトも、このテンプレートを参考にしたフォルダ構成をとっている。

## 1.1データのロード

### 1.1.1 前準備

In [1]:
# src/data/db.py より
from contextlib import contextmanager
from sqlalchemy import MetaData, create_engine
from sqlalchemy.orm import sessionmaker

# DB接続用のエンジン作成
SQLITE_DB_PATH = "../data/raw/suzuki.db"
engine = create_engine(f'sqlite:///{SQLITE_DB_PATH}')

# セッションを作成。エンジンと結びつける。
Session = sessionmaker()
Session.configure(bind=engine)

# セッションをwith句で使えるように、コンテクストとして定義。
# エラーの場合にはDBにはコミットせずにロールバックし、操作が終わったら自動的にセッションを閉じる。
@contextmanager
def session_scope():
    """Provide a transactional scope around a series of operations."""
    session = Session()
    try:
        yield session
        session.commit()
    except:
        session.rollback()
        raise
    finally:
        session.close()

In [2]:
# src/data/models.py　より

from sqlalchemy import Table

# メタデータをエンジンと結びつける
metadata = MetaData()
metadata.reflect(engine)

# テーブルデータをオブジェクトにマッピング
FTIRBaseInfo = Table('ftir_base_info', metadata, autoload=True, autoload_with=engine)
TYQ0110 = Table('TYQ0110', metadata, autoload=True, autoload_with=engine)
TYQ0120 = Table('TYQ0120', metadata, autoload=True, autoload_with=engine)
TYQ0160 = Table('TYQ0160', metadata, autoload=True, autoload_with=engine)
TYQ0210 = Table('TYQ0210', metadata, autoload=True, autoload_with=engine)
TYQ0220 = Table('TYQ0220', metadata, autoload=True, autoload_with=engine)
TYQ0650 = Table('TYQ0650', metadata, autoload=True, autoload_with=engine)
TYQ0810 = Table('TYQ0810', metadata, autoload=True, autoload_with=engine)

### 1.1.2 FTIRデータの抽出

In [3]:
# src/data/cleansing.py　より

from sqlalchemy import Numeric, and_, func, cast, or_
import pandas as pd

# DBから対象となるFTIRデータを抽出。少し時間がかかる。
with session_scope() as session:
    # ftir_base_infoテーブルから、F_ID, F_VIN, F_FAULT_PROPOSAL_LLの3列を選択する
    query = session.query(
        FTIRBaseInfo.c.F_ID,
        FTIRBaseInfo.c.F_VIN,
        FTIRBaseInfo.c.F_FAULT_PROPOSAL_LL,
    )
    
    # 上記のクエリに、国内の四輪のデータのみに絞る条件を追加
    query = query.filter(
        and_(
            FTIRBaseInfo.c.F_REPORT_COUNTRY_CODE == 'JP',
            FTIRBaseInfo.c.F_PRODUCT_SPECIFICATION == '1'
        )
    )
        
    # TYQ0210テーブルにおいて、G_RECEIVING_JUDGEMENT_CODEが1であるもののみを残す。
    query = query.join(
        TYQ0210, FTIRBaseInfo.c.F_ID == TYQ0210.c.G_ID)\
        .filter(TYQ0210.c.G_RECEIVING_JUDGEMENT_CODE == '1')
    
    # 2004年5月以降のデータのみに絞る条件を追加。年月はF_IDの3~8桁目から取得。
    ym = cast(func.substr(FTIRBaseInfo.c.F_ID, 3, 8), Numeric)
    query = query.filter(and_(ym >= 200405, ym != 299999))
    
    # データをpandasデータフレームとしてメモリ上にロード
    df = pd.read_sql(query.statement, query.session.bind)

In [4]:
df.shape

(329959, 3)

In [5]:
df.head()

,F_ID,F_VIN,F_FAULT_PROPOSAL_LL
0,JP201705B04583,MH23S-603071,エアコン作動時ヂヂヂヂ音が酷い
1,JP201705B01145,DA17W-126846,ル?ムランプがつかない
2,JP201707B02836,MK32S-198564,車検整備で入庫のため、お客様の申し出は特にない。
3,JP201705B00780,MR31S-825807,ベルト鳴き
4,JP201711B00181,MA15S-227769,急ブレーキ気味にブレーキをかけると後ろから変な音（コトコト、カラカラ）がする


In [6]:
# featherフォーマットで中間ファイルを保存しておく
INTERIM_PATH = "../data/interim/"
df.to_feather(f"{INTERIM_PATH}df.feather")

約33万件のデータを抽出できた。  
ちなみに選択に使用したクエリは以下の通り。

In [7]:
print(query.statement.compile(engine))

SELECT ftir_base_info."F_ID", ftir_base_info."F_VIN", ftir_base_info."F_FAULT_PROPOSAL_LL" 
FROM ftir_base_info JOIN "TYQ0210" ON ftir_base_info."F_ID" = "TYQ0210"."G_ID" 
WHERE ftir_base_info."F_REPORT_COUNTRY_CODE" = ? AND ftir_base_info."F_PRODUCT_SPECIFICATION" = ? AND "TYQ0210"."G_RECEIVING_JUDGEMENT_CODE" = ? AND CAST(substr(ftir_base_info."F_ID", ?, ?) AS NUMERIC) >= ? AND CAST(substr(ftir_base_info."F_ID", ?, ?) AS NUMERIC) != ?


In [8]:
# src/data/cleansing.py　より
# 点検・車検・年次〇〇関連のFTIRを除外。(F_FAULT_PROPOSAL_LL列のテキストから、それらの文字列を含むデータを除外する。)
df = df[~df['F_FAULT_PROPOSAL_LL'].str.contains(r'((点|車)検|年次)')]

/home/user01/anaconda3/envs/suzuki/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
df.shape

(312029, 3)

約2万件のデータが除外された

In [10]:
# src/data/cleansing.py　より

# 重複(文章内容が似ている)レコードの除外
# 重複除外は最後にかける。
# 他の条件で除外されるレコードと似ているレコードが余分に除外されるのを防ぐためである。

from distance import levenshtein
from itertools import combinations
from tqdm import tqdm
from typing import List


def drop_similar_str(df: pd.core.frame.DataFrame, min_distance: float,
    grp_cols: List[str] = ['F_VIN'], doc_col: str = 'F_FAULT_PROPOSAL_LL'):

    tqdm.pandas()

    # 各F_VINごとに、テキストの内容が似ている(=レーベンシュタイン距離にして0.5以下)ペアを除外する。少し時間がかかる。
    def _drop(grp):
        if grp.shape[0] == 1:
            return grp
        ids = set(grp.index)
        for id1, id2 in combinations(reversed(sorted(ids)), 2):
            if (id1 not in ids) or (id2 not in ids):
                continue
            str1 = grp[doc_col][id1]
            str2 = grp[doc_col][id2]
            if levenshtein(str1, str2, normalized=True) < min_distance:
                ids.discard(id2)
        return grp[grp.index.isin(ids)]

    return df.groupby(grp_cols, as_index=False).progress_apply(_drop).reset_index(drop=True)


similarity_rate = 0.5
df = drop_similar_str(df, similarity_rate)

100%|██████████| 276668/276668 [02:18<00:00, 2004.47it/s]


レーベンシュタイン距離は別名編集距離とも呼ばれる。[こちら](https://mieruca-ai.com/ai/levenshtein_jaro-winkler_distance/)のページが詳しい

In [11]:
df.shape

(306509, 3)

約6000件のデータが除外された

In [12]:
df.to_feather(f"{INTERIM_PATH}df_filterd.feather")

### 1.1.2 製造データ等の抽出

In [13]:
# src/data/load.py　より

# 以下の列を、ftir_base_infoテーブルから取得する
col_names = ['F_ID',
            'F_FAULT_SUBJECT_LL',
            'F_FAULT_PROPOSAL_LL',
            'F_FAULT_WHEN_LL',
            'F_FAULT_SITUATION_LL',
            'F_FAULT_WHAT_LL',
            'F_FAULT_BECAME_LL',
            'F_FAULT_CHECK_LL',
            'F_FAULT_RESULT_LL',
            'F_FAULT_CAUSE_LL',
            'F_FAULT_WHY_LL',]

with session_scope() as session:
    query = session.query(*[getattr(FTIRBaseInfo.c, col_name) for col_name in col_names])
    
    # 上記のクエリに、国内の四輪のデータのみに絞る条件を追加(追加しなくても問題ないが、量が多くなってしまう)
    query = query.filter(
        and_(
            FTIRBaseInfo.c.F_REPORT_COUNTRY_CODE == 'JP',
            FTIRBaseInfo.c.F_PRODUCT_SPECIFICATION == '1'
        )
    )
    
    df_spec = pd.read_sql(query.statement, query.session.bind)

In [14]:
print(df_spec.shape)

(368193, 11)


In [15]:
df_spec.head(3)

,F_ID,F_FAULT_SUBJECT_LL,F_FAULT_PROPOSAL_LL,F_FAULT_WHEN_LL,F_FAULT_SITUATION_LL,F_FAULT_WHAT_LL,F_FAULT_BECAME_LL,F_FAULT_CHECK_LL,F_FAULT_RESULT_LL,F_FAULT_CAUSE_LL,F_FAULT_WHY_LL
0,JP201705B00777,ガソリンメーター増減する,ガソリンメーター増減する 何か情報があればください,,,コンビネーションメーター,ガソリン満タンにして走行時１０５キロ走った時点で１メモリ減りしばらく走ると満タンになる\r\...,,,,
1,JP201705B04583,エアコン作動時ヂヂヂヂ音が酷い,エアコン作動時ヂヂヂヂ音が酷い,,,,ｴﾊﾞﾎﾟﾚｰﾀのﾄﾞﾚｰﾝからｵｲﾙ漏れあり ｺﾝﾌﾟﾚｯｻｰの圧縮不良,ｴｱｺﾝｶﾞｽ漏れ点検,ｴﾊﾞﾎﾟﾚｰﾀよりｶﾞｽ及びｵｲﾙ漏れ,,
2,JP201705B01145,ル?ムランプ不灯,ル?ムランプがつかない,,,リヤム?ルランプ,ＤＯＯＲになっていましたが点灯せず,電源点検、ア?ス点検,電源、ア?ス共に異常なし、電球を触っていたら点灯した。,ﾗﾝﾌﾟｱﾂｼ ﾘﾔﾙ-ﾑ(ｸﾞﾚ-),３回目の入庫で接点を清掃し返却しましたが再発した為、本体を交換しました。


In [16]:
# src/data/cleansing.py　より

# 先程の申出データにinner join
df_result = df.merge(df_spec)

In [17]:
for x in ["df", "df_spec", "df_result"]:print(f"{x}:{eval(x).shape}")

df:(306509, 3)
df_spec:(368193, 11)
df_result:(306509, 12)


In [18]:
df_result.head()

,F_ID,F_VIN,F_FAULT_PROPOSAL_LL,F_FAULT_SUBJECT_LL,F_FAULT_WHEN_LL,F_FAULT_SITUATION_LL,F_FAULT_WHAT_LL,F_FAULT_BECAME_LL,F_FAULT_CHECK_LL,F_FAULT_RESULT_LL,F_FAULT_CAUSE_LL,F_FAULT_WHY_LL
0,JP200703B50795,,,-,,,,,,,,
1,JP200704B60065,,冬季など気温の低い時期に始動直後から２〜３分ぐらいの間、エンジンよりカタカタ\r\n音がする...,エンジン異音,現象確認中、\r\nユ−ザ−申し出のカタカタ音は確認できずも、始動直後より１分ぐらいまでの間...,*,*,*,*,*,*,*
2,JP201802B02043,0,【特殊工具】作業効率改善の為の提案、その１,【特殊工具】作業効率改善の為の提案、その１,,,,【特殊工具】作業効率改善の為の提案、その１,,,,
3,JP201712B02574,09991-07470,特殊工具の構造について【静岡県自動車整備振興会東部支所】,ストレッチベルト組付け用特殊工具の件【整備振興会】,,,,ストレッチベルトのインストーラーが使いにくい,特殊工具の外観を比較,特殊工具の外観を撮影,,
4,JP200509B50648,100233,Air bag indicator Lamp glowing,Air bag indicator Lamp glowing,Air bag indicator Lamp glowing due to defectiv...,,,,,,,


In [19]:
import gc
del df, df_spec
gc.collect()

254

In [20]:
# src/transform/join_text.py　より
from functools import reduce
from operator import itemgetter

join_cols = ['F_FAULT_PROPOSAL_LL',
            'F_FAULT_WHEN_LL',
            'F_FAULT_SITUATION_LL']

# " $ "でjoin_colsで指定された列の文章をつなぐ関数をreduceで作成し、
# それにNAを空白で埋めたdf_resultを入力する
new_col = reduce(
    lambda a, b: a + " $ " + b,
    itemgetter(*join_cols)(df_result.fillna(''))
)

# F_IDとnew_colだけ残す。
df_result = df_result[["F_ID"]].assign(**{"DOC": new_col})

# ちなみにnew_colは以下でもかける。
"""
new_col = df_result[['F_FAULT_PROPOSAL_LL',
            'F_FAULT_WHEN_LL',
            'F_FAULT_SITUATION_LL']].\
    apply(lambda x: " $ ".join(x))
""";

In [21]:
df_result.shape

(306509, 2)

In [22]:
df_result.head()

,F_ID,DOC
0,JP200703B50795,$ $
1,JP200704B60065,冬季など気温の低い時期に始動直後から２〜３分ぐらいの間、エンジンよりカタカタ\r\n音がする...
2,JP201802B02043,【特殊工具】作業効率改善の為の提案、その１ $ $
3,JP201712B02574,特殊工具の構造について【静岡県自動車整備振興会東部支所】 $ $
4,JP200509B50648,Air bag indicator Lamp glowing $ Air bag indic...


In [23]:
del new_col
gc.collect()
df_result.to_feather(f"{INTERIM_PATH}df_filterd_joined.feather")

## 1.2 分かち書き

In [24]:
# src/transform/parse.py より

# セル内改行があるので、改行を削除する。ハイフンも一緒に削除。
# NFKCを用いてUnicodeの正規化を行う。
target_texts = (df_result.DOC
    .str.replace('\r', '')
    .str.replace('\n', '')
    .str.replace('ｰ', 'ー')
    .str.normalize('NFKC')
)

Unicodeの正規化については[こちら](http://tech.albert2005.co.jp/501/)のブログがわかりやすい

In [25]:
# MeCabで分かち書き
import MeCab
_TAGGER = MeCab.Tagger("-Ochasen")

# テスト
parsed_test = _TAGGER.parse("今日は良い天気です")
print(parsed_test)
parsed_test

今日	キョウ	今日	名詞-副詞可能		
は	ハ	は	助詞-係助詞		
良い	ヨイ	良い	形容詞-自立	形容詞・アウオ段	基本形
天気	テンキ	天気	名詞-一般		
です	デス	です	助動詞	特殊・デス	基本形
EOS



'今日\tキョウ\t今日\t名詞-副詞可能\t\t\nは\tハ\tは\t助詞-係助詞\t\t\n良い\tヨイ\t良い\t形容詞-自立\t形容詞・アウオ段\t基本形\n天気\tテンキ\t天気\t名詞-一般\t\t\nです\tデス\tです\t助動詞\t特殊・デス\t基本形\nEOS\n'

MeCabのアルゴリズムの説明は、作成者である工藤氏の[こちらの資料](http://www.jtpa.org/wp-content/uploads/2014/06/MeCab.pdf)がわかりやすい

In [26]:
# src/transform/parse.py より

# 今回はMeCabで分かち書き。少し時間がかかる。
import MeCab
_TAGGER = MeCab.Tagger("-Ochasen")

# 改行で分割し、末尾のタブ2つを除去
parsed_texts = (
        target_texts
        .apply(_TAGGER.parse)
        .str.split('\n')
        .apply(lambda l: l[:-2])
        .tolist()
    )

In [27]:
[x[:5] for x in parsed_texts[:5]]

[['$\t$\t$\t記号-一般\t\t', '$\t$\t$\t記号-一般\t\t'],
 ['冬季\tトウキ\t冬季\t名詞-一般\t\t',
  'など\tナド\tなど\t助詞-副助詞\t\t',
  '気温\tキオン\t気温\t名詞-一般\t\t',
  'の\tノ\tの\t助詞-格助詞-一般\t\t',
  '低い\tヒクイ\t低い\t形容詞-自立\t形容詞・アウオ段\t基本形'],
 ['【\t【\t【\t記号-括弧開\t\t',
  '特殊\tトクシュ\t特殊\t名詞-形容動詞語幹\t\t',
  '工具\tコウグ\t工具\t名詞-一般\t\t',
  '】\t】\t】\t記号-括弧閉\t\t',
  '作業\tサギョウ\t作業\t名詞-サ変接続\t\t'],
 ['特殊\tトクシュ\t特殊\t名詞-形容動詞語幹\t\t',
  '工具\tコウグ\t工具\t名詞-一般\t\t',
  'の\tノ\tの\t助詞-連体化\t\t',
  '構造\tコウゾウ\t構造\t名詞-一般\t\t',
  'について\tニツイテ\tについて\t助詞-格助詞-連語\t\t'],
 ['Air\tエア\tAir\t名詞-固有名詞-一般\t\t',
  'bag\tbag\tbag\t名詞-一般\t\t',
  'indicator\tindicator\tindicator\t名詞-一般\t\t',
  'Lamp\tランプ\tLamp\t名詞-固有名詞-人名-一般\t\t',
  'glowing\tglowing\tglowing\t名詞-固有名詞-組織\t\t']]

In [28]:
# src/transform/parse.py より
import itertools

# list(itertools.chain.from_iterable(~~))は、入れ子のリスト(=リストのリスト)をつないだ、通常のリストを返す
parsed_texts_series = pd.Series(list(itertools.chain.from_iterable(parsed_texts)))

# parseされた文章の単語数
n_words = list(map(len, parsed_texts))

# parseされた単語の位置
positions = pd.Series(list(itertools.chain.from_iterable(map(range, n_words))))

# 含まれる単語の数だけ同じ文章を繰り返したseriesを用意
target_texts_column = pd.Series(
    list(itertools.chain.from_iterable(
            itertools.starmap(
                itertools.repeat,
                zip(target_texts, n_words)))))

# 横に結合
df_parse = pd.concat(
        [target_texts_column, positions, parsed_texts_series.str.split('\t', expand=True)],
        axis=1)

In [29]:
# 参考: series.str.splitでexpand=Trueとすると、列方向に広がったdata frameが返ってくる
parsed_texts_series[:5].str.split('\t', expand=True)

,0,1,2,3,4,5
0,$,$,$,記号-一般,,
1,$,$,$,記号-一般,,
2,冬季,トウキ,冬季,名詞-一般,,
3,など,ナド,など,助詞-副助詞,,
4,気温,キオン,気温,名詞-一般,,


In [30]:
df_parse.head()

,0,1,0,1,2,3,4,5
0,$ $,0,$,$,$,記号-一般,,
1,$ $,1,$,$,$,記号-一般,,
2,冬季など気温の低い時期に始動直後から2〜3分ぐらいの間、エンジンよりカタカタ音がする。気温の...,0,冬季,トウキ,冬季,名詞-一般,,
3,冬季など気温の低い時期に始動直後から2〜3分ぐらいの間、エンジンよりカタカタ音がする。気温の...,1,など,ナド,など,助詞-副助詞,,
4,冬季など気温の低い時期に始動直後から2〜3分ぐらいの間、エンジンよりカタカタ音がする。気温の...,2,気温,キオン,気温,名詞-一般,,


In [31]:
del parsed_texts, target_texts, parsed_texts_series, target_texts_column, positions
gc.collect()

27

In [32]:
# src/transform/filter_word_types.py より
# 列名を変更
df_parse.columns = [
    'original_text',
    'word_position',
    'surface_form',
    'pronunciation',
    'base_form',
    'word_class',
    'inflection_type',
    'inflection_form',
]

# 文章の単語数の数だけ、F_IDを繰り返したseriesを用意
id_column = pd.Series(
    list(itertools.chain.from_iterable(
            itertools.starmap(
                itertools.repeat,
                zip(df_result.F_ID, n_words)))))

# 上記のID列と、parseして作った先ほどのdataframeを結合
df_parse = pd.concat([id_column.rename('id'), df_parse], axis=1).set_index(['id', 'word_position'])

In [33]:
df_parse.head()

original_text  \
id             word_position                                                      
JP200703B50795 0                                                          $  $    
               1                                                          $  $    
JP200704B60065 0              冬季など気温の低い時期に始動直後から2〜3分ぐらいの間、エンジンよりカタカタ音がする。気温の...   
               1              冬季など気温の低い時期に始動直後から2〜3分ぐらいの間、エンジンよりカタカタ音がする。気温の...   
               2              冬季など気温の低い時期に始動直後から2〜3分ぐらいの間、エンジンよりカタカタ音がする。気温の...   

                             surface_form pronunciation base_form word_class  \
id             word_position                                                   
JP200703B50795 0                        $             $         $      記号-一般   
               1                        $             $         $      記号-一般   
JP200704B60065 0                       冬季           トウキ        冬季      名詞-一般   
               1                       など            ナド        など     助詞-副助詞   
               2                       気温           キオン        気温      名詞-一般   

                             inflection_type inflection_form  
id             word_position                                  
JP200703B50795 0                                              
               1                                              
JP200704B60065 0                                              
               1                                              
               2

In [34]:
# df_parse.reset_index().to_feather(f"{INTERIM_PATH}df_parse.feather")

In [35]:
import numpy as np
np.array(sorted(df_parse.word_class.unique()))

array(['その他-間投', 'フィラー', '副詞-一般', '副詞-助詞類接続', '助動詞', '助詞-並立助詞', '助詞-係助詞',
       '助詞-副助詞', '助詞-副助詞／並立助詞／終助詞', '助詞-副詞化', '助詞-接続助詞', '助詞-格助詞-一般',
       '助詞-格助詞-引用', '助詞-格助詞-連語', '助詞-特殊', '助詞-終助詞', '助詞-連体化', '動詞-接尾',
       '動詞-自立', '動詞-非自立', '名詞-サ変接続', '名詞-ナイ形容詞語幹', '名詞-一般', '名詞-代名詞-一般',
       '名詞-代名詞-縮約', '名詞-副詞可能', '名詞-動詞非自立的', '名詞-固有名詞-一般', '名詞-固有名詞-人名-一般',
       '名詞-固有名詞-人名-名', '名詞-固有名詞-人名-姓', '名詞-固有名詞-地域-一般', '名詞-固有名詞-地域-国',
       '名詞-固有名詞-組織', '名詞-引用文字列', '名詞-形容動詞語幹', '名詞-接尾-サ変接続', '名詞-接尾-一般',
       '名詞-接尾-人名', '名詞-接尾-副詞可能', '名詞-接尾-助動詞語幹', '名詞-接尾-助数詞', '名詞-接尾-地域',
       '名詞-接尾-形容動詞語幹', '名詞-接尾-特殊', '名詞-接続詞的', '名詞-数', '名詞-特殊-助動詞語幹',
       '名詞-非自立-一般', '名詞-非自立-副詞可能', '名詞-非自立-助動詞語幹', '名詞-非自立-形容動詞語幹',
       '形容詞-接尾', '形容詞-自立', '形容詞-非自立', '感動詞', '接続詞', '接頭詞-動詞接続',
       '接頭詞-名詞接続', '接頭詞-数接続', '記号-アルファベット', '記号-一般', '記号-句点', '記号-括弧閉',
       '記号-括弧開', '記号-読点', '連体詞'], dtype='<U15')

In [36]:
del df_result
gc.collect()

13

In [37]:
# src/transform/filter_word_types.py より
# '名詞-数'以外の名詞または未知語の単語をフラグ付け
df_parse = df_parse.assign(
    used=df_parse.word_class.str.match('^(名詞[^数]*|未知語)$')
)

In [38]:
df_parse.head()

original_text  \
id             word_position                                                      
JP200703B50795 0                                                          $  $    
               1                                                          $  $    
JP200704B60065 0              冬季など気温の低い時期に始動直後から2〜3分ぐらいの間、エンジンよりカタカタ音がする。気温の...   
               1              冬季など気温の低い時期に始動直後から2〜3分ぐらいの間、エンジンよりカタカタ音がする。気温の...   
               2              冬季など気温の低い時期に始動直後から2〜3分ぐらいの間、エンジンよりカタカタ音がする。気温の...   

                             surface_form pronunciation base_form word_class  \
id             word_position                                                   
JP200703B50795 0                        $             $         $      記号-一般   
               1                        $             $         $      記号-一般   
JP200704B60065 0                       冬季           トウキ        冬季      名詞-一般   
               1                       など            ナド        など     助詞-副助詞   
               2                       気温           キオン        気温      名詞-一般   

                             inflection_type inflection_form   used  
id             word_position                                         
JP200703B50795 0                                              False  
               1                                              False  
JP200704B60065 0                                               True  
               1                                              False  
               2                                               True

In [39]:
# 今回の分析に使用する単語の種類一覧
df_used_word_types = pd.read_csv("../data/external/used_word_types.csv", 
                                 encoding="utf-16", sep="\t")
df_used_word_types

,0,1
0,連体詞,NaN
1,名詞,サ変接続
2,名詞,ナイ形容詞語幹
3,名詞,形容動詞語幹
4,名詞,動詞非自立的
5,名詞,副詞可能
6,名詞,一般
7,名詞,数
8,名詞,固有名詞
9,動詞,自立


In [40]:
# src/transform/filter_word_types.py より

# join用の列を作成
df_parse = pd.concat(
    [df_parse, df_parse.word_class.str.split('-', expand=True)],
    axis=1,
)

# 列名を文字列型に変更
df_parse.columns = list(map(str, df_parse.columns))
df_parse.head()

original_text  \
id             word_position                                                      
JP200703B50795 0                                                          $  $    
               1                                                          $  $    
JP200704B60065 0              冬季など気温の低い時期に始動直後から2〜3分ぐらいの間、エンジンよりカタカタ音がする。気温の...   
               1              冬季など気温の低い時期に始動直後から2〜3分ぐらいの間、エンジンよりカタカタ音がする。気温の...   
               2              冬季など気温の低い時期に始動直後から2〜3分ぐらいの間、エンジンよりカタカタ音がする。気温の...   

                             surface_form pronunciation base_form word_class  \
id             word_position                                                   
JP200703B50795 0                        $             $         $      記号-一般   
               1                        $             $         $      記号-一般   
JP200704B60065 0                       冬季           トウキ        冬季      名詞-一般   
               1                       など            ナド        など     助詞-副助詞   
               2                       気温           キオン        気温      名詞-一般   

                             inflection_type inflection_form   used   0    1  \
id             word_position                                                   
JP200703B50795 0                                              False  記号   一般   
               1                                              False  記号   一般   
JP200704B60065 0                                               True  名詞   一般   
               1                                              False  助詞  副助詞   
               2                                               True  名詞   一般   

                                 2     3  
id             word_position              
JP200703B50795 0              None  None  
               1              None  None  
JP200704B60065 0              None  None  
               1              None  None  
               2              None  None

In [41]:
# src/transform/filter_word_types.py より

# df_used_word_typesとinner join
df_parse = (
    df_parse
    .reset_index()
    .merge(df_used_word_types)
    .sort_values(['id', 'word_position'])
    .assign(used=True)
)
df_parse.head()

,id,word_position,original_text,surface_form,pronunciation,base_form,word_class,inflection_type,inflection_form,used,0,1,2,3
43652,JP200410B50001,0,高速走行中(トンネルの中)、車両前部より白煙(あるいは水蒸気)シートを起こしたところ炎を確認...,高速,コウソク,高速,名詞-一般,,,True,名詞,一般,None,None
1686445,JP200410B50001,1,高速走行中(トンネルの中)、車両前部より白煙(あるいは水蒸気)シートを起こしたところ炎を確認...,走行中,ソウコウチュウ,走行中,名詞-固有名詞-一般,,,True,名詞,固有名詞,一般,None
43653,JP200410B50001,3,高速走行中(トンネルの中)、車両前部より白煙(あるいは水蒸気)シートを起こしたところ炎を確認...,トンネル,トンネル,トンネル,名詞-一般,,,True,名詞,一般,None,None
43654,JP200410B50001,7,高速走行中(トンネルの中)、車両前部より白煙(あるいは水蒸気)シートを起こしたところ炎を確認...,車両,シャリョウ,車両,名詞-一般,,,True,名詞,一般,None,None
43655,JP200410B50001,8,高速走行中(トンネルの中)、車両前部より白煙(あるいは水蒸気)シートを起こしたところ炎を確認...,前部,ゼンブ,前部,名詞-一般,,,True,名詞,一般,None,None


In [42]:
df_parse.to_csv(f"{INTERIM_PATH}df_parse_filterd.csv", index=False)